### Classify ship images using CNN

In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os

C:\Amrendra\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
os.chdir("D:\My Personal Documents\Learnings\Data Science\Hackathan - Game of Deep Learning\\train")

In [3]:
train = pd.read_csv('train.csv')

In [ ]:
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('images/'+train['image'][i], target_size=(128,128,3), grayscale=False)
    img = image.img_to_array(img)
    #img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|████████████████████████████████████████████████████████████████████████████| 12504/12504 [03:31<00:00, 59.21it/s]


In [ ]:
X.shape

In [ ]:
y=train['category'].values
#y = to_categorical(y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
from matplotlib import pyplot
from scipy.misc import toimage
def show_imgs(X):
    pyplot.figure(1)
    k = 0
    for i in range(0,4):
        for j in range(0,4):
            pyplot.subplot2grid((4,4),(i,j))
            pyplot.imshow(toimage(X[k]))
            k = k+1
    # show the plot
    pyplot.show()
 
show_imgs(x_test[:16])

In [ ]:
x_train=x_train/255
x_test = x_test/255


In [ ]:
n_class=6
y_train = keras.utils.to_categorical(y_train,n_class)
y_test = keras.utils.to_categorical(y_test,n_class)

In [ ]:
n_conv = 64
k_conv = (3,3)

In [ ]:
y_train.shape

In [ ]:
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32,(3,3),padding='same', activation='relu',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(MaxPooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(64,(3,3),padding='same', activation='relu',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(MaxPooling2D())
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(128,(3,3), padding='same',activation='relu',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(MaxPooling2D())
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(6,activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=5,verbose=1, validation_data=(x_test,y_test))

In [ ]:
test = pd.read_csv('test_ApKoW4T.csv')

In [ ]:
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('images/'+test['image'][i], target_size=(28,28,3), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

In [ ]:
prediction = model.predict_classes(test)

In [ ]:
pd.DataFrame(prediction).to_csv('test_pred.csv')